# CROP-Seq pipeline

A bit of set-up

In [1]:
from Templates.tools.features import *
from Templates.tools import datasetMaker
import Templates.tools.DatasetDB as DB
from gpauth import GPAuth
import os
import anndata as ad 
import pandas as pd
import numpy as np
import scanpy as sc

### Please enter your password to authenticate

In [2]:
auth = GPAuth()

Authenticating User: 'ghaffars'


 ········


Successfully authenticated.


### Export Settings

In [2]:
Notebooks_HOME = "./Templates/"   # the directory that contains input notebooks
A = os.getcwd()
Notebooks_Savepath = os.path.abspath(os.path.join(A ,"../Reports"))  # the directory that contains exported reports
Reports_format = "notebook" # the output report format can be one of "notebook", "html" and "pdf" 
Dataset_HOME = "/gstore/scratch/u/ghaffars/Dataset/crc_NGS5704/" # the directory that contains all AnnData files stored on the disk

### Dataset Info

In [3]:
dsdbs = ['DS000016632', 'DS000016634', 'DS000016633', 'DS000016631']

### Test dataset creation

In [6]:
DS_test = False  #A boolean based on whether your want to create a test dataset to run the pipeline on 

### Run this section only if you want to create a test dataset from your original dataset and run a test instead on working on the original adatset 

In [7]:
#if DS_test:
#    test_DatasetID, version = datasetMaker.Dataset_maker(DatasetID, DEV)

In [8]:
#test_DatasetID = 'test-DS000006669'

### Define metadata parameters

In [9]:
# any identifiers you want to attach to the experiment, usually GEO ids, publication etc.
sources = [{"id": "Siavash-1234", "name": "Geo-ID"}]
#Authors names
author = "SG"

### Define QC transcriptome parameters

In [10]:
organism = 'human' # human or mouse
groupby = 'Sample' # For tabulation
min_cells = 3
min_genes = 500
max_genes = None
pct_mt = None
total_counts = None
n_top_genes = 2000
ribo = True
doublets = True # This refers to hashing. Only for hashing when you have demux_type in obs
# create a dictionary to keep the parameters organized
gene_param = dict({'organism':organism, 'groupby':groupby, 'min_cells':min_cells, 'min_genes':min_genes,
                  'max_genes':max_genes, 'pct_mt':pct_mt, 'total_counts':total_counts, 'n_top_genes':n_top_genes,
                  'ribo':ribo, 'doublets':doublets})

### Define QC Hashing Parameters

In [12]:
#experiment = 'hashing'
topBarcodesToPlot = 5
bottomBarcodesToPlot = 5
fix_barcodes=False
valid_assignments=None
# create a dictionary to keep the parameters organized
hashing_param = dict({'topBarcodesToPlot':topBarcodesToPlot, 'bottomBarcodesToPlot':bottomBarcodesToPlot, 
                                       'fix_barcodes':fix_barcodes, 'valid_assignments':valid_assignments})

### Define QC Crispr Parameters

In [13]:
#experiment = 'crispr'
topBarcodesToPlot = 5
bottomBarcodesToPlot = 5
fix_barcodes=False
valid_assignments=None
# create a dictionary to keep the parameters organized
crispr_param = dict({'topBarcodesToPlot':topBarcodesToPlot, 'bottomBarcodesToPlot':bottomBarcodesToPlot, 
                                       'fix_barcodes':fix_barcodes, 'valid_assignments':valid_assignments})

### Dataset Harmonization for all the split runs

In [20]:
%%capture
for i in dsdbs:
    DatasetID = i
    DEV = False
    DF = Features(DatasetID, auth, DEV = DEV,
              DS_test = DS_test,
                sources = sources, 
                 author = author,
              Notebooks_HOME = Notebooks_HOME,
             Notebooks_Savepath = Notebooks_Savepath,
             Reports_format = Reports_format,
             Dataset_HOME = Dataset_HOME,
                 gene_param = gene_param,
              hashing_param = hashing_param,
              crispr_param = crispr_param,)
    #DF.Dataset_Harmonization()
    DF.QC_Transcriptome()
    #DF.QC_Hashing()
    #DF.QC_Crispr()
    

In [4]:
adatas = []

for i in dsdbs:
    DSID = i
    path = f'{Dataset_HOME}/{DSID}'
    file = os.path.abspath(os.path.join(path ,"raw_qc.h5ad"))
    adata = sc.read_h5ad(file)
    adata = adata[adata.obs["qc_pass"]==True].copy()
    
    adatas.append(adata)
    
    

In [5]:
adatas

[AnnData object with n_obs × n_vars = 39583 × 36603
     obs: 'Sample', 'Barcode', 'DemuxType_crispr', 'DemuxAssignment_crispr', 'DemuxType_hashing', 'DemuxAssignment_hashing', 'cellline', 'timepoint', 'HTO', 'NGS_ID', 'Biological_replicate', '10Xrun', 'sublibrary', 'gRNA_library_MOI', 'gene_symbol', 'class', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'qc_pass', 'S_score', 'G2M_score', 'phase'
     var: 'ID', 'Symbol', 'Type', 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
     uns: '.internal'
     layers: 'counts',
 AnnData object with n_obs × n_vars = 35847 × 36603
     obs: 'Sample', 'Barcode', 'DemuxType_crispr', 'DemuxAssignment_crispr', 'DemuxType_hashing', 'DemuxAssignment_hashing', 'cellline', 'timepoint', 'HTO', 'NGS_ID', 'Biological_replicate', '10Xrun', 'sublibrary', 'gRNA_library_MOI', 'gene_symbol', 'class', 'n_genes_by_counts', 'total_counts', 'total_counts_mt

In [6]:
concat = ad.concat(adatas,join='outer')

In [7]:
concat

AnnData object with n_obs × n_vars = 147916 × 36603
    obs: 'Sample', 'Barcode', 'DemuxType_crispr', 'DemuxAssignment_crispr', 'DemuxType_hashing', 'DemuxAssignment_hashing', 'cellline', 'timepoint', 'HTO', 'NGS_ID', 'Biological_replicate', '10Xrun', 'sublibrary', 'gRNA_library_MOI', 'gene_symbol', 'class', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'qc_pass', 'S_score', 'G2M_score', 'phase'
    layers: 'counts'

In [8]:
concat.var["Symbol"] = concat.var.index.copy()

In [9]:
concat.var

,Symbol
3xLinker,3xLinker
MIR1302-2HG,MIR1302-2HG
FAM138A,FAM138A
OR4F5,OR4F5
AL627309.1,AL627309.1
...,...
AC023491.2,AC023491.2
AC007325.1,AC007325.1
AC007325.4,AC007325.4
AC007325.2,AC007325.2


In [10]:
concat

AnnData object with n_obs × n_vars = 147916 × 36603
    obs: 'Sample', 'Barcode', 'DemuxType_crispr', 'DemuxAssignment_crispr', 'DemuxType_hashing', 'DemuxAssignment_hashing', 'cellline', 'timepoint', 'HTO', 'NGS_ID', 'Biological_replicate', '10Xrun', 'sublibrary', 'gRNA_library_MOI', 'gene_symbol', 'class', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'qc_pass', 'S_score', 'G2M_score', 'phase'
    var: 'Symbol'
    layers: 'counts'

In [11]:
concat.obs['gene_symbol'] = np.where(concat.obs['gene_symbol']=='unknown', 
                                    concat.obs['DemuxAssignment_crispr'].apply(lambda x:x.split('_')[0]),concat.obs['gene_symbol']) 

In [21]:
#sc.pp.filter_genes(concat, min_cells=3)

In [39]:
concat.write_h5ad("/gstore/scratch/u/ghaffars/Dataset/crc_NGS5704/concat_filtered.h5ad")

## Combine with the rest of sublib4 i.e. NGS5570

In [12]:
concat_2 = sc.read_h5ad("/gstore/scratch/u/ghaffars/Dataset/crc_NGS5570/concat_filtered.h5ad")

In [ ]:
concat_2

In [ ]:
concat_2.obs['NGS_ID'] = 'NGS5570'

In [ ]:
concat_2.obs['gene_symbol'] = np.where(concat_2.obs['gene_symbol']=='unknown', 
                                    concat_2.obs['DemuxAssignment_crispr'].apply(lambda x:x.split('_')[0]),concat_2.obs['gene_symbol']) 

In [16]:
concat_2.obs[concat_2.obs['gene_symbol']=='unknown']

,Sample,Barcode,DemuxType_crispr,DemuxAssignment_crispr,DemuxType_hashing,DemuxAssignment_hashing,cellline,timepoint,HTO,NGS_ID,...,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_ribo,pct_counts_ribo,qc_pass,S_score,G2M_score,phase


In [17]:
concat_2.write("/gstore/scratch/u/ghaffars/Dataset/crc_NGS5570/concat_filtered.h5ad")

In [13]:
concat_sublib4 = ad.concat([concat,concat_2],join='outer')

In [14]:
concat_sublib4

AnnData object with n_obs × n_vars = 446413 × 36603
    obs: 'Sample', 'Barcode', 'DemuxType_crispr', 'DemuxAssignment_crispr', 'DemuxType_hashing', 'DemuxAssignment_hashing', 'cellline', 'timepoint', 'HTO', 'NGS_ID', 'Biological_replicate', '10Xrun', 'sublibrary', 'gRNA_library_MOI', 'gene_symbol', 'class', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'qc_pass', 'S_score', 'G2M_score', 'phase'
    layers: 'counts'

In [15]:
concat_sublib4.var["Symbol"] = concat_sublib4.var.index.copy()

In [18]:
concat_sublib4.write("/gstore/scratch/u/ghaffars/Dataset/sublib4/raw_qc.h5ad")

## Upload to DSDB

In [ ]:
concat_sublib4= sc.read("/gstore/scratch/u/ghaffars/Dataset/sublib4/raw_qc.h5ad")

In [3]:
#concat_sublib4= ad.read_h5ad("/gstore/scratch/u/ghaffars/Dataset/sublib4/raw_qc.h5ad")

In [4]:
concat_sublib4

AnnData object with n_obs × n_vars = 446413 × 36603
    obs: 'Sample', 'Barcode', 'DemuxType_crispr', 'DemuxAssignment_crispr', 'DemuxType_hashing', 'DemuxAssignment_hashing', 'cellline', 'timepoint', 'HTO', 'NGS_ID', 'Biological_replicate', '10Xrun', 'sublibrary', 'gRNA_library_MOI', 'gene_symbol', 'class', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'qc_pass', 'S_score', 'G2M_score', 'phase'
    var: 'Symbol'
    layers: 'counts'

In [5]:
adatas_updated={}

In [6]:
new_experiment = "raw_qc"
DEV = False
title = "Recursion DLD1 Library4 Screen Day 5- combination of NGS 5570 and NGS 5704"
description = "Production scale screen in DLD-1 cells with 1/4 of the genome-wide CRISPR library (sub-lib-4, ~22K guides). 1 time point: Day-5. We performed 12 rxns of 10x 3' HT kit with an estimated loading of 52K cells.We performed a pilot study to test the library quality before the production-level sequencing(NGS5703). We will submit 36 libraries (12 GEX, 12 HTO and 12 sgRNA) We estimate needing - 600M per GEX, - 20M per HTO and - 50M per sgRNA libraries now that we have analyzed the QC run. This run is to add additional cells needed for analysis."
name_space = [{"id": "GRCh38", "type": "genome"}]
organism = f"human"
sources = [{"id": "sublib4", "name": "Recursion DLD1"}]
tech_name = "scRNA-seq"
author = "SG"

In [7]:
import pydsdb
import multiassayexperiment as mae
import singlecellexperiment as sce
import pandas as pd
from gpauth import GPAuth

In [8]:
from multiassayexperiment import makeMAE

In [9]:
sExpt = sce.fromAnnData(concat_sublib4)

In [10]:
print(sExpt)

Class SingleCellExperiment with 36603 features and 446413 cells 
  mainExperimentName: None 
  assays: ['counts', 'X'] 
  features: Index(['Symbol'], dtype='object') 
  cell annotations: Index(['Sample', 'Barcode', 'DemuxType_crispr', 'DemuxAssignment_crispr',
       'DemuxType_hashing', 'DemuxAssignment_hashing', 'cellline', 'timepoint',
       'HTO', 'NGS_ID', 'Biological_replicate', '10Xrun', 'sublibrary',
       'gRNA_library_MOI', 'gene_symbol', 'class', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo',
       'pct_counts_ribo', 'qc_pass', 'S_score', 'G2M_score', 'phase'],
      dtype='object') 
  reduced dimensions: None 
  alternative experiments: None


In [11]:
scr_metadata = pydsdb.create_scr_metadata(
    description=description,
    name_space=name_space,
    organism=organism,
    sources=sources,
    technology_name=tech_name,
    title=title,
    # other optional metadata
)

In [12]:
pydsdb.add_metadata(sExpt, scr_metadata)

In [13]:
# prepare dataset level metadata
dataset_metadata = pydsdb.create_dataset_metadata(
    description=("Recursion DLD1 Library4 Screen Day 5- combination of NGS 5570 and NGS 5704"),
    title="Production scale screen in DLD-1 cells with 1/4 of the genome-wide CRISPR library (sub-lib-4, ~22K guides). 1 time point: Day-5. We performed 12 rxns of 10x 3' HT kit with an estimated loading of 52K cells.We performed a pilot study to test the library quality before the production-level sequencing(NGS5703). We will submit 36 libraries (12 GEX, 12 HTO and 12 sgRNA) We estimate needing - 600M per GEX, - 20M per HTO and - 50M per sgRNA libraries now that we have analyzed the QC run. This run is to add additional cells needed for analysis.",
    authors="ghaffars",
)

In [14]:
obj = makeMAE({"raw_qc": sExpt})

/apps/user/gpy/envs/dev/GPy39/lib/python3.9/site-packages/multiassayexperiment/MultiAssayExperiment.py:127: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for group, rows in agroups:


In [15]:
# attach dataset level metadata
pydsdb.add_metadata(obj, dataset_metadata)

In [16]:
upload = pydsdb.Upload(obj,file_size_limit = "64 GiB")

In [17]:
permissions = pydsdb.create_permissions_info()

In [18]:
dsid, version = upload.submit(auth,permissions, dev=DEV, test=False, mode="sts:boto3")

11:47:44 -> Collating and validating project resources.
11:47:44 -> Attempting upload.


100%|████████████████████████████████████████| 50/50 [06:15<00:00,  7.52s/file] 


Clearing upload info.
11:54:05 -> Upload completed.
Processing...
Processing...
Processing...
Upload successful, your DSID is 'DS000016647' and version is: '1'. Also written to 'upload_info.json' in staging dir.
